### Install necessary libraries


In [ ]:
pip install numpy

In [ ]:
! pip install instaloader matplotlib numpy


##### Load Session using sessionID to login 

In [264]:

import instaloader
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from datetime import datetime
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
 
def load_instagram_session(session_id):

    try:
        loader = instaloader.Instaloader()
        loader.context._session.cookies.set("sessionid", session_id)
        return loader
    except Exception as e:
        print(f"Error loading session: {e}")
        return None


#### Fetch and display basic Instagram profile details 

In [265]:
def fetch_profile_data(username, loader):

    try:
        profile = instaloader.Profile.from_username(loader.context, username)
        print(f"\n--- Profile Information ---")
        print(f"Name: {profile.full_name}")
        print(f"Username: {profile.username}")
        print(f"Bio: {profile.biography}")
        print(f"Followers: {profile.followers}")
        print(f"Following: {profile.followees}")
        print(f"Media Count: {profile.mediacount}")
        print(f"Is Verified: {'Yes' if profile.is_verified else 'No'}")
        print(f"Is Private: {'Yes' if profile.is_private else 'No'}")
        return profile
    except Exception as e:
        print(f"Error fetching profile data: {e}")
        return None

#### Hashtag Categorization 

In [266]:


# Predefined hashtag categories
HASHTAG_CATEGORIES = {
    "Fashion": ["fashion", "style", "ootd", "trend", "vintage", "designer", "streetwear", "couture", "runway"],
    "Beauty": ["beauty", "makeup", "skincare", "cosmetics", "beautytips", "haircare", "nails", "beautyblogger", "glam","loreal"],
    "Fitness": ["fitness", "workout", "health", "gym", "exercise", "fitfam", "wellness", "yoga", "strength", "cardio"],
    "Travel": ["travel", "wanderlust", "adventure", "vacation", "destination", "explore", "travelgram", "luxurytravel", "traveller"],
    "Food": ["food", "foodie", "yum", "recipe", "cooking", "foodstagram", "delicious", "chef", "homemade", "baking"],
    "Technology": ["tech", "gadgets", "innovation", "startup", "coding", "programming", "AI", "blockchain", "software"],
    "Lifestyle": ["lifestyle", "home", "decor", "minimalism", "living", "interior", "selfcare", "hygge", "motivation"],
    "Education": ["education", "learning", "study", "knowledge", "school", "onlinelearning", "books", "career", "student"],
    "Entertainment": ["entertainment", "movies", "music", "celebrity", "netflix", "concert", "tvshows", "popculture", "theater"],
    "Parenting": ["parenting", "momlife", "dadlife", "baby", "kids", "family", "parenthood", "newborn", "parentingtips"],
    "Pets": ["pets", "dogs", "cats", "animal", "puppy", "kitten", "petlovers", "petstagram", "dogsofinstagram"],
    "Finance": ["finance", "investing", "money", "budget", "crypto", "economy", "wealth", "savings", "financialfreedom"],
    "Gaming": ["gaming", "esports", "gamer", "videogames", "streaming", "playstation", "xbox", "pcgaming", "gameon"],
    "Photography": ["photography", "photooftheday", "naturephotography", "landscape", "portrait", "camera", "photoshoot", "travelphotography"],
    "Art": ["art", "artist", "painting", "illustration", "creative", "drawing", "design", "artwork", "digitalart"],
    "Business": ["business", "marketing", "branding", "entrepreneur", "startup", "ecommerce", "leadership", "success"],

}

def categorize_influencer_by_hashtags(profile):
    try:
        hashtags = []
        for post in profile.get_posts():
            if (datetime.now() - post.date).days <= 180:
                hashtags.extend(post.caption_hashtags)
                if len(hashtags) >= 50: 
                    break

        hashtag_counter = Counter(hashtags)

        category_count = {category: 0 for category in HASHTAG_CATEGORIES}
        for hashtag, count in hashtag_counter.items():
            for category, keywords in HASHTAG_CATEGORIES.items():
                if hashtag.lower() in keywords:
                    category_count[category] += count

       
        influencer_type = max(category_count, key=category_count.get)
        print(f"Categorized as a '{influencer_type}' influencer.")

        plot_hashtag_frequency(hashtag_counter)
        plot_category_distribution(category_count)

    except Exception as e:
        print(f"Error during categorization: {e}")

def plot_hashtag_frequency(hashtag_counter):
   
    common_hashtags = hashtag_counter.most_common(10)
    hashtags, counts = zip(*common_hashtags)
    plt.figure(figsize=(10, 6))
    plt.barh(hashtags, counts, color='skyblue')
    plt.xlabel('Frequency')
    plt.title('Top 10 Most Frequent Hashtags')
    plt.gca().invert_yaxis()
    plt.show()

def plot_category_distribution(category_count):
   
    categories = list(category_count.keys())
    counts = list(category_count.values())
    plt.figure(figsize=(10, 6))
    plt.bar(categories, counts, color='purple', alpha=0.7)
    plt.xlabel("Categories")
    plt.ylabel("Hashtag Count")
    plt.title("Hashtag Categorization for Influencer")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


#### Engagement Rate Analysis

In [267]:

def calculate_engagement_rate(likes, comments, followers):
    try:
        total_engagement = sum(likes) + sum(comments)
        engagement_rate = (total_engagement / (followers * len(likes))) * 100
        return engagement_rate
    except ZeroDivisionError:
        return 0

def analyze_engagement_rate(profile):
    try:
        followers = profile.followers
        likes, comments, post_dates = [], [], []

        for post in profile.get_posts():
            if (datetime.now() - post.date).days <= 180:
                likes.append(post.likes)
                comments.append(post.comments)
                post_dates.append(post.date.strftime('%Y-%m'))
            else:
                break

        engagement_rates = {}
        for date, like, comment in zip(post_dates, likes, comments):
            if date not in engagement_rates:
                engagement_rates[date] = {'likes': 0, 'comments': 0, 'count': 0}
            engagement_rates[date]['likes'] += like
            engagement_rates[date]['comments'] += comment
            engagement_rates[date]['count'] += 1

        monthly_engagement_rate = {}
        for date, data in engagement_rates.items():
            total_engagement = data['likes'] + data['comments']
            monthly_engagement_rate[date] = (total_engagement / (followers * data['count'])) * 100

        for month, rate in monthly_engagement_rate.items():
            print(f"Engagement Rate for {month}: {rate:.2f}%"
                  )
        plot_engagement_rate(monthly_engagement_rate)

        return monthly_engagement_rate
    except Exception as e:
        print(f"Error calculating engagement rate: {e}")
        return None

def plot_engagement_rate(monthly_engagement_rate):
    months = list(monthly_engagement_rate.keys())
    rates = list(monthly_engagement_rate.values())

    plt.figure(figsize=(10, 6))
    plt.plot(months, rates, marker='o', color='blue', label="Engagement Rate (%)")
    plt.xlabel("Month")
    plt.ylabel("Engagement Rate (%)")
    plt.title("Engagement Rate Over Last 6 Months")
    plt.grid()
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


### Earning estimation

In [268]:
import matplotlib.pyplot as plt
from datetime import datetime
import instaloader


def estimate_earnings(followers, engagement_rate, rate_per_1000=0.10):
    if followers <= 0 or engagement_rate <= 0:
        return 0 

    estimated_earnings = ((followers * engagement_rate) / 1000) * rate_per_1000
    return estimated_earnings

def analyze_engagement_rate(profile):
    try:
        followers = profile.followers
        likes, comments, post_dates = [], [], []

        for post in profile.get_posts():
            if (datetime.now() - post.date).days <= 180:
                likes.append(post.likes)
                comments.append(post.comments)
                post_dates.append(post.date.strftime('%Y-%m'))
            else:
                break

        engagement_rates = {}
        for date, like, comment in zip(post_dates, likes, comments):
            if date not in engagement_rates:
                engagement_rates[date] = {'likes': 0, 'comments': 0, 'count': 0}
            engagement_rates[date]['likes'] += like
            engagement_rates[date]['comments'] += comment
            engagement_rates[date]['count'] += 1

        monthly_engagement_rate = {}
        for date, data in engagement_rates.items():
            total_engagement = data['likes'] + data['comments']
            monthly_engagement_rate[date] = (total_engagement / (followers * data['count'])) * 100

        for month, rate in monthly_engagement_rate.items():
            print(f"Engagement Rate for {month}: {rate:.2f}%")

        plot_engagement_rate(monthly_engagement_rate)

        return monthly_engagement_rate
    except Exception as e:
        print(f"Error calculating engagement rate: {e}")
        return None

def plot_engagement_rate(monthly_engagement_rate):
    months = list(monthly_engagement_rate.keys())
    rates = list(monthly_engagement_rate.values())

    plt.figure(figsize=(10, 6))
    plt.plot(months, rates, marker='o', color='blue', label="Engagement Rate (%)")
    plt.xlabel("Month")
    plt.ylabel("Engagement Rate (%)")
    plt.title("Engagement Rate Over Last 6 Months")
    plt.grid()
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_estimated_earnings(monthly_engagement_rates, followers, rate_per_1000=0.10):

    months = list(monthly_engagement_rates.keys())
    engagement_rates = list(monthly_engagement_rates.values())
    earnings = [estimate_earnings(followers, rate, rate_per_1000) for rate in engagement_rates]

    plt.figure(figsize=(10, 6))
    plt.bar(months, earnings, color='green', alpha=0.7, label="Earnings ($)")
    plt.xlabel("Month")
    plt.ylabel("Estimated Earnings ($)")
    plt.title("Estimated Earnings Per Month (Last 6 Months)")
    plt.grid(axis='y')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


#### Follower Growth Rate

In [261]:
def calculate_growth_rate(follower_counts):
    growth_rates = {}
    months = list(follower_counts.keys())
    for i in range(1, len(months)):
        prev_count = follower_counts[months[i - 1]]
        current_count = follower_counts[months[i]]
        if prev_count > 0:
            growth_rates[months[i]] = ((current_count - prev_count) / prev_count) * 100
        else:
            growth_rates[months[i]] = 0
    return growth_rates



def plot_growth_rate(follower_counts):
    months = list(follower_counts.keys())
    follower_data = list(follower_counts.values())
    growth_rates = calculate_growth_rate(follower_counts)

    plt.figure(figsize=(10, 6))
    plt.plot(months, follower_data, marker='o', color='blue', label='Follower Count')
    plt.xlabel("Month")
    plt.ylabel("Follower Count")
    plt.title("Follower Growth Over Last 3 Months")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend(loc="upper left")
    plt.twinx()
    growth_data = list(growth_rates.values())
    plt.plot(months[1:], growth_data, marker='x', color='red', label='Growth Rate (%)')
    plt.ylabel("Growth Rate (%)")
    plt.legend(loc="upper right")
    plt.tight_layout()
    plt.show()



#### Collaboration Rate


In [269]:
import matplotlib.pyplot as plt

TOP_BRANDS = [
    'LOréal', 'Myntra', 'Nykaa', 'Mamaearth', 'Biotique', 'Lakmé', 'Sugar Cosmetics',
    'The Body Shop', 'Forest Essentials', 'Wow Skin Science', 'Huda Beauty', 'Sephora',
    'Aveda', 'Clinique', 'Maybelline', 'Revlon', 'MAC Cosmetics', 'Kiehl\'s', 'Tata Harper',
    'Chanel', 'Estee Lauder', 'Bobbi Brown', 'Fenty Beauty', 'Nivea', 'Ponds', 'Vichy', 'Olay',
    'Neutrogena', 'St. Ives', 'Bioderma', 'Lush', 'Innisfree', 'Shiseido', 'Dove', 'Plum',
    'Santoor', 'Garnier', 'Himalaya', 'Cleansing Balm', 'Vaseline', 'Biore', 'Kama Ayurveda',
    'SUGAR', 'Colorbar', 'L\'Occitane', 'Sephora India', 'Plum Goodness', 'Nykaa Beauty',
    'Tata Cliq', 'Koovs', 'Chumbak', 'Biba', 'FabIndia', 'Ritu Kumar', 'Jabong', 'Zara',
    'H&M', 'Mango', 'Puma', 'Adidas', 'Nike', 'Reebok', 'Fastrack', 'Tanishq', 'CaratLane', 
    'Kalyan Jewellers', 'Croma', 'Bajaj Finserv', 'Myntra', 'Wipro', 'Bata', 'Lenskart', 'Café Coffee Day', 
    'Patanjali', 'Haldiram', 'PepsiCo', 'CocaCola', 'Tata Motors', 'Ford', 'Mahindra', 'Hyundai', 'Suzuki', 
    'Maruti', 'Volkswagen', 'Royal Enfield', 'Hero MotoCorp', 'Honda', 'Ford', 'BMW', 'Toyota', 'Nissan', 
    'Acer', 'Dell', 'Lenovo', 'HP', 'Apple', 'Samsung', 'Xiaomi', 'OnePlus', 'Realme', 'Oppo', 'Vivo'
]

def calculate_collaboration_rate(username, profile):

    total_posts = 0
    collaboration_posts = 0


    collaboration_rate_over_time = []


    for post in profile.get_posts():
        total_posts += 1
        
        has_collab_hashtags = any(brand.lower() in post.caption.lower() for brand in TOP_BRANDS)
        
        has_collab_tags = any(tagged_user.lower() in [brand.lower() for brand in TOP_BRANDS] for tagged_user in post.tagged_users)

        if has_collab_hashtags or has_collab_tags:
            collaboration_posts += 1
        
        collaboration_rate = (collaboration_posts / total_posts) * 100 if total_posts > 0 else 0
        return collaboration_rate


In [ ]:
def instagram_analysis():

    session_id = input("Enter your Instagram session ID: ")
    username = input("Enter the Instagram username to analyze: ")
    
    loader = load_instagram_session(session_id)
    if not loader:
        return
    
    profile = fetch_profile_data(username, loader)
    if not profile:
        return
    
    print("\n--- Hashtag Categorization ---")
    categorize_influencer_by_hashtags(profile)


    print("\n--- Engagement Rate Analysis ---")
    monthly_engagement_rate = analyze_engagement_rate(profile)
    if monthly_engagement_rate:
        print("Engagement Rates:", monthly_engagement_rate)

    print("\n--- Estimated Earnings Analysis ---")
    followers = profile.followers
    plot_estimated_earnings(monthly_engagement_rate, followers)

    print("\n--- Follower growth ---")
    follower_counts = {follower.date.strftime('%Y-%m'): follower.followers for follower in profile.get_followers()}
    plot_growth_rate(follower_counts)

    print("\n--- Collaboration Rate Analysis ---")
    collaboration_rate = calculate_collaboration_rate(username, profile)
    print(f"Collaboration Rate: {collaboration_rate:.2f}%")
   
instagram_analysis()
